<a href="https://colab.research.google.com/github/Tasnima158/clip/blob/main/clip.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import files
files.upload()


Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"tasnimaahmed","key":"8dba801f027d63f06be93c0f4d59fb00"}'}

In [ ]:
!ls /content


kaggle.json  sample_data


In [ ]:
# !mkdir -p ~/.kaggle
# !cp kaggle.json ~/.kaggle/
# !chmod 600 ~/.kaggle/kaggle.json


In [ ]:
!pip install kagglehub


In [ ]:
!ls $path


kaggle.json  sample_data


In [ ]:
import kagglehub

path = kagglehub.dataset_download("paultimothymooney/chest-xray-pneumonia")

print("Path to dataset files:", path)

Using Colab cache for faster access to the 'chest-xray-pneumonia' dataset.
Path to dataset files: /kaggle/input/chest-xray-pneumonia


#  Install CLIP

In [ ]:
!pip install ftfy regex tqdm
!pip install git+https://github.com/openai/CLIP.git


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.8/44.8 kB 3.1 MB/s eta 0:00:00
  Cloning https://github.com/openai/CLIP.git to /tmp/pip-req-build-synypv6f
  Running command git clone --filter=blob:none --quiet https://github.com/openai/CLIP.git /tmp/pip-req-build-synypv6f
  Resolved https://github.com/openai/CLIP.git to commit dcba3cb2e2827b402d2701e7e1c7d9fed8a20ef1
  Preparing metadata (setup.py) ... done
  Created wheel for clip: filename=clip-1.0-py3-none-any.whl size=1369490 sha256=987d8ced524692529e53abb2fdbb8c05394b94e68511120f68fd5e682495dc8e
  Stored in directory: /tmp/pip-ephem-wheel-cache-peoancrb/wheels/35/3e/df/3d24cbfb3b6a06f17a2bfd7d1138900d4365d9028aa8f6e92f
Successfully built clip


# libraries & load pretrained CLIP

In [ ]:
import torch
import clip
from PIL import Image
import os

device = "cuda" if torch.cuda.is_available() else "cpu"
model, preprocess = clip.load("ViT-B/32", device=device)

print("CLIP loaded on:", device)


100%|███████████████████████████████████████| 338M/338M [00:05<00:00, 65.3MiB/s]


CLIP loaded on: cuda


# Define ZERO-SHOT text prompts


In [ ]:
class_prompts = [
    "a chest X-ray showing pneumonia",
    "a normal chest X-ray"
]

text_tokens = clip.tokenize(class_prompts).to(device)


# Load TEST images

In [ ]:
test_dir = os.path.join(path, "chest_xray", "test")

image_paths = []
true_labels = []

for label in ["PNEUMONIA", "NORMAL"]:
    folder = os.path.join(test_dir, label)
    for img in os.listdir(folder):
        image_paths.append(os.path.join(folder, img))
        true_labels.append(label)

print("Total test images:", len(image_paths))


Total test images: 624


# ZERO-SHOT

In [ ]:
correct = 0

with torch.no_grad():
    text_features = model.encode_text(text_tokens) # 2,512
    text_features /= text_features.norm(dim=-1, keepdim=True) #(2, 77, 512)

    for img_path, true_label in zip(image_paths, true_labels):
        image = preprocess(Image.open(img_path).convert("RGB")) \
                .unsqueeze(0).to(device)

        image_features = model.encode_image(image)
        image_features /= image_features.norm(dim=-1, keepdim=True)

        similarity = (image_features @ text_features.T).softmax(dim=-1)
        pred_index = similarity.argmax().item()

        predicted_label = "PNEUMONIA" if pred_index == 0 else "NORMAL"

        if predicted_label == true_label:
            correct += 1

zero_shot_accuracy = correct / len(image_paths)
zero_shot_accuracy


0.625

# FINE-TUNING CLIP (CUSTOM MODEL)

# Text descriptions

In [ ]:
class_text = {
    "PNEUMONIA": [
        "a chest X-ray showing pneumonia",
        "a chest X-ray with lung infection",
        "a chest X-ray with abnormal lung opacity"
    ],
    "NORMAL": [
        "a normal chest X-ray",
        "a healthy chest X-ray",
        "a chest X-ray with clear lungs"
    ]
}


# Create a PyTorch Dataset

In [ ]:
import random

class ChestXrayDataset(Dataset):
    def __init__(self, root_dir):
        self.samples = []
        for label in ["PNEUMONIA", "NORMAL"]:
            folder = os.path.join(root_dir, label)
            for img in os.listdir(folder):
                self.samples.append((os.path.join(folder, img), label))

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        img_path, label = self.samples[idx]
        image = preprocess(Image.open(img_path).convert("RGB"))

        text = random.choice(class_text[label])
        text = clip.tokenize(text)[0]

        label_id = 0 if label == "PNEUMONIA" else 1
        return image, text, label_id


# Create DataLoader

In [ ]:
train_dir = os.path.join(path, "chest_xray", "train")

train_dataset = ChestXrayDataset(train_dir)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

print("Training samples:", len(train_dataset))


Training samples: 5216


# Freeze most of CLIP

In [ ]:
for param in model.transformer.parameters():
    param.requires_grad = False

In [ ]:
for param in model.visual.parameters():
    param.requires_grad = True

# Optimizer & loss

In [ ]:
optimizer = torch.optim.AdamW(model.visual.parameters(), lr=3e-5)
loss_fn = torch.nn.CrossEntropyLoss()


# Fine-tuning loop

In [ ]:
with torch.no_grad():
    class_embeddings = {}

    for label, prompts in class_text.items():
        tokens = clip.tokenize(prompts).to(device)
        feats = model.encode_text(tokens)
        feats /= feats.norm(dim=-1, keepdim=True)
        class_embeddings[label] = feats.mean(dim=0)


# Evaluate fine-tuned CLIP

In [ ]:
model.train()
epochs = 5

for epoch in range(epochs):
    total_loss = 0

    for images, _, labels in train_loader:
        images = images.to(device)
        labels = labels.to(device)

        image_features = model.encode_image(images)
        image_features = image_features / image_features.norm(dim=-1, keepdim=True)

        sim_p = image_features @ class_embeddings["PNEUMONIA"]
        sim_n = image_features @ class_embeddings["NORMAL"]

        logits = torch.stack([sim_p, sim_n], dim=1)

        loss = loss_fn(logits, labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    print(f"Epoch {epoch+1}, Loss: {total_loss/len(train_loader):.4f}")


Epoch 1, Loss: nan
Epoch 2, Loss: nan
Epoch 3, Loss: nan
Epoch 4, Loss: nan
Epoch 5, Loss: nan


In [ ]:
test_dir = os.path.join(path, "chest_xray", "test")

image_paths = []
true_labels = []

for label in ["PNEUMONIA", "NORMAL"]:
    folder = os.path.join(test_dir, label)
    for img in os.listdir(folder):
        image_paths.append(os.path.join(folder, img))
        true_labels.append(label)


In [ ]:
model.eval()
correct = 0

with torch.no_grad():
    for img_path, true_label in zip(image_paths, true_labels):
        image = preprocess(Image.open(img_path).convert("RGB")).unsqueeze(0).to(device)

        image_features = model.encode_image(image)
        image_features /= image_features.norm(dim=-1, keepdim=True)

        sim_p = torch.dot(image_features.squeeze(), class_embeddings["PNEUMONIA"])
        sim_n = torch.dot(image_features.squeeze(), class_embeddings["NORMAL"])

        predicted_label = "PNEUMONIA" if sim_p > sim_n else "NORMAL"

        if predicted_label == true_label:
            correct += 1

finetuned_accuracy = correct / len(image_paths)
finetuned_accuracy


0.375